# Train A Model (Basic)

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import lightning.pytorch as pl

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
        
    def forward(self, x):
        return self.l1(x)
    
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))
        
    def forward(self, x):
        return self.l1(x)

In [ ]:
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.save_hyperparameters()
        self.encoder = encoder
        self.decoder = decoder
        
    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        
        val_loss = F.mse_loss(x_hat, x)
        
        self.log("val_loss", val_loss)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = F.mse_loss(x_hat, x)
        
        self.log("test_loss", test_loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
train_set = MNIST(os.getcwd(), download=True, train=True, transform=transforms.ToTensor())
test_set = MNIST(os.getcwd(), download=True, train=False, transform=transforms.ToTensor())

In [ ]:
train_loader = DataLoader(train_set, batch_size=512)
test_loader = DataLoader(test_set, batch_size=512)

### Trainer for training

In [ ]:
autoencoder = LitAutoEncoder(Encoder(), Decoder())

In [ ]:
trainer = pl.Trainer(max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=vali)

### Trainer for Testing

In [ ]:
trainer.test(model=autoencoder, dataloaders=test_loader)

### Add validation loop

In [ ]:
autoencoder = LitAutoEncoder(Encoder(), Decoder())

In [ ]:
train_set_size = int(len(train_set) * 0.8)
valid_set_size = len(train_set) - train_set_size

seed = torch.Generator().manual_seed(42)
train_set, valid_set = random_split(train_set, [train_set_size, valid_set_size], generator = seed)

In [ ]:
train_loader = DataLoader(train_set, batch_size = 512)
valid_loader = DataLoader(valid_set, batch_size = 512)

In [ ]:
# Disable checkpointing
# trainer = pl.Trainer(max_epochs=1, default_root_dir="GAN/", enable_checkpointing=False)

In [ ]:
early_stop_callback = pl.callbacks.early_stopping.EarlyStopping(monitor="val_loss", mode="min")

In [ ]:
# to customize early stopping criteria
# early_stop_callback = pl.callbacks.early_stopping.EarlyStopping(monitor="vall_accuracy", min_delta=0.00, patience=3, verbose=False, mode="max")

In [ ]:
trainer = pl.Trainer(max_epochs=1, 
                     default_root_dir="GAN/",
                     callbacks=[early_stop_callback],
                     enable_checkpointing=True)
trainer.fit(autoencoder, train_loader, valid_loader)

In [ ]:
trainer.test(autoencoder, test_loader)

### LightningModule from Checkpoint

In [ ]:
model = LitAutoEncoder.load_from_checkpoint("GAN/lightning_logs/version_0/checkpoints/epoch=0-step=94.ckpt",encoder=Encoder(), decoder=Decoder())

In [ ]:
model.eval()

### Resume training state

In [ ]:
model = LitAutoEncoder(encoder=Encoder(), decoder=Decoder())

In [ ]:
trainer = pl.Trainer(max_epochs=1, default_root_dir="GAN/")

In [ ]:
trainer.fit(autoencoder,
            train_loader,
            valid_loader,
            ckpt_path="GAN/lightning_logs/version_1/checkpoints/epoch=0-step=94.ckpt")